<a href="https://colab.research.google.com/github/nicsim22/DS110-Content/blob/main/Lecture30SQL_nosol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

*There was just one source of data that Cynthia hadn't tamed yet.  Facebook Marketplace was a huge source of transactions that were probably conducted in cash.  But their transactions were stored in a database with a SQL interface, and SAGE hadn't touched it yet.*

*Psssh.  How hard could learning this "SQL" be, compared to everything else Cynthia had done?  She started looking up how to make queries.*

# What is SQL?

SQL (Structured Query Language) is a language for **interacting with databases**.  Databases store information in an organized way, and the **queries to access the data are often written in SQL**.  This is especially true if the database uses a Relational Database Management System (RDBMS), which treats the data as being organized in tables.

Even for big data, where before it was considered too expensive to do a full RDBMS system with SQL queries, things have gradually moved in the direction of the greater organization necessary to make these queries.  So even when dealing with big data, the interface will often be similar to SQL.

SQL has multiple implementations, but the one we'll be demonstrating is SQLite.  This python module gives us the power to make SQL queries against an existing database and put the result in a Pandas dataframe.  (These examples use the books.db file (using a file as a pretend database) and corresponding examples from the textbook.)

An interesting aspect of SQL is that a query specifies what it wants, but not how to get it.  SQLite compiles the query into a program, and chooses among multiple approaches to get the data (for example, what to filter by first when handling an AND).  As a result, what the engine is doing is often treated like a black box by the user.

## Basic setup and column queries

In [ ]:
# For Google colab only
from google.colab import files

uploaded = files.upload() # Get "books.db" file

Saving books.db to books.db


In [ ]:
import sqlite3
connection = sqlite3.connect('books.db') #create connection obj

import pandas as pd
pd.read_sql('SELECT * FROM authors', connection) #panda has a real SQL function #string w all capital letters is SQL
#select all columns from authors table --> has id, first, and last column in authors table
#will send sequel command to database

,id,first,last
0,1,Paul,Deitel
1,2,Harvey,Deitel
2,3,Abbey,Deitel
3,4,Dan,Quirk
4,5,Alexander,Wald


Here, we use SQLite to interact with a file, treating it as the database.  .connect() creates a Connection object, which can then be used as an argument indicating where our database is to be found.  And finally, the string with all the capital letters is SQL, indicating that we want all (*) lines from the authors table.  The result of the read_sql is a DataFrame containing the results of the query.



Note that some SQL systems require a semicolon at the end of each statement, particularly if they accept multiple statements at a time.  But this implementation doesn't.  There are a variety of little differences between SQL implementations that vary from vendor to vendor.

Also note that the caps-lock is a stylistic choice rather than absolutely essential - the queries in SQLite would work as well with different case choices.  But capitalizing keywords makes it easier to see what's happening.

This particular "authors" table is kind of small.  The same database has a "titles" table with more columns.

In [ ]:
pd.read_sql('SELECT * from titles', connection) #select all columns in titles table, connect to database

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


To select data from just some of the columns, we can list the desired columns where we put a * before.

In [ ]:
pd.read_sql('SELECT title, edition from titles', connection) #no * bc not selecting all columns, only want title, edition from titles table

,title,edition
0,Intro to Python for CS and DS,1
1,Internet & WWW How to Program,5
2,Java How to Program,11
3,C How to Program,8
4,Visual Basic 2012 How to Program,6
5,Visual C# How to Program,6
6,Visual C++ How to Program,2
7,C++ How to Program,10
8,Android How to Program,3
9,Android 6 for Programmers,3


If you're trying to just see the different values for a particular column,
**SELECT DISTINCT** avoids repeating itself.

In [ ]:
pd.read_sql('SELECT DISTINCT copyright FROM titles', connection) #only one column (e.g. copyright) --> use SELECT DISTINCT

,copyright
0,2020
1,2012
2,2018
3,2016
4,2014
5,2017
6,2008


These are all dataframes being returned by the queries, of course, so we could call dataframe functions on whatever was returned.

In [ ]:
df = pd.read_sql('SELECT edition from titles', connection)
print(df)
df.mean()

   edition
0        1
1        5
2       11
3        8
4        6
5        6
6        2
7       10
8        3
9        3


,0
edition,5.5


## Where, Like, and Order By

The **WHERE** clause allows **filtering** of values. We'll also move to **triple-quote syntax** as the queries get long.


In [ ]:
pd.read_sql("""SELECT title, edition from titles WHERE edition > 5""", connection)

,title,edition
0,Java How to Program,11
1,C How to Program,8
2,Visual Basic 2012 How to Program,6
3,Visual C# How to Program,6
4,C++ How to Program,10


Other numerical operators include **<, <=, =, >=, =, and != (for not equal)**.  The conditions can be joined with **AND, OR, and NOT**.  (Also note that as queries get long,
SQL doesn't mind additional whitespace, including newlines.)

In [ ]:
pd.read_sql("""SELECT title, edition
from titles WHERE edition != 6 AND edition >= 3
AND NOT edition >= 10""", connection)

,title,edition
0,Internet & WWW How to Program,5
1,C How to Program,8
2,Android How to Program,3
3,Android 6 for Programmers,3


Note that sometimes columns with numbers are **mistyped** in the database **as being strings**, in which case, c**omparison operators won't work** as expected. To fix this, **cast the column as an integer** before making the comparison, like this:

In [ ]:
pd.read_sql("""SELECT title, edition from titles WHERE cast(edition as int) > 5""", connection)

,title,edition
0,Java How to Program,11
1,C How to Program,8
2,Visual Basic 2012 How to Program,6
3,Visual C# How to Program,6
4,C++ How to Program,10


There isn't any missing data in these tables, but you could also avoid missing entries with **WHERE [columnname] IS NOT NULL**.  (Null is the value for missing data in SQL.)

WHERE can also look for **regular-expression-like patterns using LIKE**.  Here we retrieve all the Deitels' books about how to program.  The **% acts like a** * in other regular expressions, matching zero or more characters.

In [ ]:
pd.read_sql("""SELECT title, edition from titles
            WHERE title LIKE '%How to Program'""", connection) #select all from title, edition column where title column contains "How to Program"

,title,edition
0,Internet & WWW How to Program,5
1,Java How to Program,11
2,C How to Program,8
3,Visual Basic 2012 How to Program,6
4,Visual C# How to Program,6
5,Visual C++ How to Program,2
6,C++ How to Program,10
7,Android How to Program,3


WHERE also understands **NOT LIKE**, which gets all the other entries.

In [ ]:
pd.read_sql("""SELECT title, edition FROM titles
            WHERE title NOT LIKE '%How to Program'""", connection) #Not like --> do not have "How to Program" in their title

,title,edition
0,Intro to Python for CS and DS,1
1,Android 6 for Programmers,3


When there are many results, it may matter how they're ordered.  **ORDER BY [column] [ASC/DESC]** will choose the column and whether it's ascending or descending order.  The **default is ascending.**

In [ ]:
pd.read_sql("""SELECT title, edition FROM titles
               ORDER BY edition DESC""", connection)

,title,edition
0,Java How to Program,11
1,C++ How to Program,10
2,C How to Program,8
3,Visual Basic 2012 How to Program,6
4,Visual C# How to Program,6
5,Internet & WWW How to Program,5
6,Android How to Program,3
7,Android 6 for Programmers,3
8,Visual C++ How to Program,2
9,Intro to Python for CS and DS,1


In [ ]:
pd.read_sql("""SELECT title, edition from titles
               ORDER BY edition""", connection) # No DESC

,title,edition
0,Intro to Python for CS and DS,1
1,Visual C++ How to Program,2
2,Android How to Program,3
3,Android 6 for Programmers,3
4,Internet & WWW How to Program,5
5,Visual Basic 2012 How to Program,6
6,Visual C# How to Program,6
7,C How to Program,8
8,C++ How to Program,10
9,Java How to Program,11


**LIMIT [number]** finds just the **top [number] entries** in the table, which is useful when dealing with a large table.  It's particularly useful in conjunction with ORDER BY.

In [ ]:
pd.read_sql("""SELECT title, edition FROM titles
              ORDER BY edition DESC LIMIT 3""", connection)

,title,edition
0,Java How to Program,11
1,C++ How to Program,10
2,C How to Program,8


## Quick checks

How would you create a DataFrame that had just the books that start with "Android" from the titles table?

In [ ]:
pd.read_sql("SELECT * FROM titles WHERE title LIKE 'Android%'", connection) #want to select all titles so * not titles

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0134289366,Android 6 for Programmers,3,2016


What about selecting titles published in or after 2017 that are at least 3rd edition?

In [ ]:
pd.read_sql('SELECT * from titles WHERE copyright >= 2017 AND edition >= 3', connection)

,isbn,title,edition,copyright
0,0134743350,Java How to Program,11,2018
1,0134601548,Visual C# How to Program,6,2017
2,0134448235,C++ How to Program,10,2017
3,0134444302,Android How to Program,3,2017


And the 4 smallest edition numbers?

In [ ]:
pd.read_sql("SELECT * FROM titles ORDER BY edition limit 4", connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0136151574,Visual C++ How to Program,2,2008
2,0134444302,Android How to Program,3,2017
3,0134289366,Android 6 for Programmers,3,2016


## Joins

Now we'll take a look at joins, starting with the small author_ISBN table.  (ISBNs are codes that uniquely identify books.)

In [ ]:
pd.read_sql("SELECT * FROM author_ISBN", connection) #each id number represents a specific author --> foreign keys that hook numbers up to the actl author

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673
5,1,0132151006
6,2,0132151006
7,3,0132151006
8,1,0134743350
9,2,0134743350


The provided author_ISBN table is somewhat cryptic as-is, but its id column refers to an author from the author table.  (This makes the id in the author_ISBN table a "foreign key."  In the table where it uniquely identifies authors, the id is the "primary key.")  The query can "join" multiple tables and use this key to match the data.

In [ ]:
pd.read_sql("""SELECT first, last, isbn
               FROM authors
               INNER JOIN author_ISBN  #combine 2 columns authors and author_ISBN
                  ON authors.id = author_ISBN.id
               ORDER BY last,first""", connection)

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006
5,Harvey,Deitel,0134743350
6,Harvey,Deitel,0133976890
7,Harvey,Deitel,0133406954
8,Harvey,Deitel,0134601548
9,Harvey,Deitel,0136151574


Notice that isbn was not in the authors table, but we could include it because we were performing the INNER JOIN.  There are also a lot more entries here than in the authors table, which only listed each author once.  The database's use of tables helps it avoid redundancy in how it's using memory.

We can join across multiple tables in this way, but if columns appear in multiple tables, like isbn here, we have to be careful to specify which table the columns belong to, as in the SELECT below.  Otherwise, we'll get an error saying the query was ambiguous.

In [ ]:
pd.read_sql("""SELECT first, last, titles.isbn, title
               FROM authors
               INNER JOIN author_ISBN
                  ON authors.id = author_ISBN.id #match these two columns
               INNER JOIN titles
                  ON author_ISBN.isbn = titles.isbn #have author_ISBN match titles
               ORDER BY last,first""", connection)

ProgrammingError: Cannot operate on a closed database.

Besides INNER JOIN, SQL supports a few other joins that differ in how they treat missing data.  In an **INNER JOIN, only records with keys in both tables are joined**.  In a **FULL JOIN**, aka FULL OUTER JOIN, all records are surfaced, but **null values are given for missing data when one side is present and the other is not**.  A** LEFT JOIN or a RIGHT JOIN** privileges the first or second table, showing all entries for that table and giving null values when the other table doesn't have a match.  On the whole, **INNER JOIN is probably the most useful (only show results where there is data from both tables).**

# Inserting, changing, and deleting data

It's also possible to **insert data into the table with SQL**.  The object that does this is called a **cursor**, and it can generally be used to interact with the database in specific places.  An **INSERT INTO can add a line**, while an **UPDATE can update the values of a particular row**.

In [ ]:
cursor = connection.cursor()
cursor = cursor.execute("""INSERT INTO authors (first, last)
                           VALUES ('John', 'Doe')""")

pd.read_sql("SELECT * from authors", connection)

ProgrammingError: Cannot operate on a closed database.

In [ ]:
cursor = cursor.execute("""UPDATE authors SET first='Jane' WHERE first='John' AND last='Doe'""")
pd.read_sql("SELECT * from authors", connection)

ProgrammingError: Cannot operate on a closed database.

You can also delete entries from the database with DELETE FROM.

In [ ]:
cursor = cursor.execute("DELETE FROM authors WHERE first='Dan'")
pd.read_sql("SELECT * from authors", connection)

,id,first,last
0,1,Paul,Deitel
1,2,Harvey,Deitel
2,3,Abbey,Deitel
3,5,Alexander,Wald
4,6,Jane,Doe


Out of all of these commands, SELECT and WHERE are the commands you're most likely to interact with, pulling the data you want and then manipulating the data on your own end within the DataFrame.  These useful basic operations are also the ones you'd be likely to use in a big distributed database.

When you're done with the SQL connection, you should close it.

In [ ]:
connection.close() #rmb to close SQL connection!

# Dremel, BigQuery, Drill

Google uses a SQL interface to its logs, called Dremel.  It made available a public version of Dremel roughly 10 years ago, and Apache's version of it is called Drill.  The queries in these systems use standard SQL with only slight changes, but the search for the answer to the query is distributed across many servers, allowing a very parallel search for the query results.  The speed requires several optimizations in how the data is stored, such as storing the columns together in storage instead of storing complete records together.  But the user only needs to know the SQL commands to successfully use it.

While it seemed at first as if Big Data might not use SQL, big players like Google use it to interact with their data in a structured way.